# 30. Verificación de la notación indicial de las ecuaciones de compatibilidad de Saint-Venant

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Juan Nicolás Ramírez, <jnramirezg@unal.edu.co> | 2023-02-23 | Creando código|
|Diego Andrés Álvarez Marín, <daalvarez@unal.edu.co> | 2023-02-23 | Creando código|


A partir de la siguiente ecuación en notación indicial:

$$\varepsilon_{ij,km}+\varepsilon_{mk,ji}-\varepsilon_{ik,jm}-\varepsilon_{mj,ki}=0$$

Se busca extraer las 6 ecuaciones de compatibilidad de Saint-Venant.

Importamos la librería necesaria:

In [1]:
from sympy import *

Se definen las variables de posición:

In [2]:
x_ = symbols("x, y, z")

Creamos la matriz de deformaciones como una matriz en la que todos sus elementos dependen de $(x,y,z)$

In [3]:
e = zeros(3)
for i in range(3):
    for j in range(3):
        if i==j:
            e[i,j] = Function(f"varepsilon_{x_[i]}")(*x_)
        elif i<j:
            e[i,j] = e[j,i] = Rational(1/2)*Function(f"gamma_{x_[i]}{x_[j]}")(*x_)

In [4]:
e

Matrix([
[varepsilon_x(x, y, z),   gamma_xy(x, y, z)/2,   gamma_xz(x, y, z)/2],
[  gamma_xy(x, y, z)/2, varepsilon_y(x, y, z),   gamma_yz(x, y, z)/2],
[  gamma_xz(x, y, z)/2,   gamma_yz(x, y, z)/2, varepsilon_z(x, y, z)]])

Creamos una lista para almacenar las ecuaciones a calcular:

In [5]:
ecu = []

Calculamos las ecuaciones: $\varepsilon_{ij,km}+\varepsilon_{mk,ji}-\varepsilon_{ik,jm}-\varepsilon_{mj,ki}=0$

In [6]:
for i in range(3):
    for j in range(3):
        for k in range(3):
            for m in range(3):
                res = simplify(+ e[i,j].diff(x_[k], x_[m])
                               + e[m,k].diff(x_[j], x_[i])
                               - e[i,k].diff(x_[j], x_[m])
                               - e[m,j].diff(x_[k], x_[i]))
                if res != 0:         # Se almacena la ecuación si es diferente de cero.
                    ecu.append(res)

In [7]:
eqs = Matrix(ecu)  # Convertimos la lista a formato Matrix de sympy.

In [8]:
eqs.shape  # Calculamos cuántas ecuaciones quedaron almacenadas.

(36, 1)

$\varepsilon_{ij,km}+\varepsilon_{mk,ji}-\varepsilon_{ik,jm}-\varepsilon_{mj,ki}=0$ al tener cuatro (4) subindices, tiene $3⁴=81$ ecuaciones posibles, pero después de descartar las que son cero (0), quedan solo 36.

Convertimos las ecuaciones repetidas a cero:

In [9]:
for i in range(len(eqs)):
    if eqs[i] == 0: 
        continue
    for j in range(i+1, len(eqs)):
        if eqs[j] == 0: 
            continue
        if simplify(eqs[i] + eqs[j]) == 0 or simplify(eqs[i] - eqs[j]) == 0:
            eqs[j] = 0

In [10]:
eqs

Matrix([
[                                       Derivative(varepsilon_x(x, y, z), (y, 2)) + Derivative(varepsilon_y(x, y, z), (x, 2)) - Derivative(gamma_xy(x, y, z), x, y)],
[ Derivative(gamma_yz(x, y, z), (x, 2))/2 - Derivative(gamma_xy(x, y, z), x, z)/2 - Derivative(gamma_xz(x, y, z), x, y)/2 + Derivative(varepsilon_x(x, y, z), y, z)],
[                                                                                                                                                                 0],
[                                       Derivative(varepsilon_x(x, y, z), (z, 2)) + Derivative(varepsilon_z(x, y, z), (x, 2)) - Derivative(gamma_xz(x, y, z), x, z)],
[                                                                                                                                                                 0],
[                                                                                                                                                                

Eliminamos las ecuaciones repetidas que anteriormente convertimos en ceros:

In [11]:
eqs = Matrix([ i for i in eqs if i != 0 ])

In [12]:
eqs

Matrix([
[                                       Derivative(varepsilon_x(x, y, z), (y, 2)) + Derivative(varepsilon_y(x, y, z), (x, 2)) - Derivative(gamma_xy(x, y, z), x, y)],
[ Derivative(gamma_yz(x, y, z), (x, 2))/2 - Derivative(gamma_xy(x, y, z), x, z)/2 - Derivative(gamma_xz(x, y, z), x, y)/2 + Derivative(varepsilon_x(x, y, z), y, z)],
[                                       Derivative(varepsilon_x(x, y, z), (z, 2)) + Derivative(varepsilon_z(x, y, z), (x, 2)) - Derivative(gamma_xz(x, y, z), x, z)],
[-Derivative(gamma_xz(x, y, z), (y, 2))/2 + Derivative(gamma_xy(x, y, z), y, z)/2 + Derivative(gamma_yz(x, y, z), x, y)/2 - Derivative(varepsilon_y(x, y, z), x, z)],
[ Derivative(gamma_xy(x, y, z), (z, 2))/2 - Derivative(gamma_xz(x, y, z), y, z)/2 - Derivative(gamma_yz(x, y, z), x, z)/2 + Derivative(varepsilon_z(x, y, z), x, y)],
[                                       Derivative(varepsilon_y(x, y, z), (z, 2)) + Derivative(varepsilon_z(x, y, z), (y, 2)) - Derivative(gamma_yz(x, y, z), y, 

Finalmente, reorganizando llegamos a las siguientes seis (6) ecuacione, que son las de compatibilidad de Saint-Venant:

$$\displaystyle\frac{\partial² \varepsilon_x}{\partial y²} + \frac{\partial² \varepsilon_y}{\partial x²}=\frac{\partial²\gamma_{xy}}{\partial x \partial y}$$
$$\displaystyle\frac{\partial² \varepsilon_y}{\partial z²} + \frac{\partial² \varepsilon_z}{\partial y²}=\frac{\partial²\gamma_{yz}}{\partial y \partial z}$$
$$\displaystyle\frac{\partial² \varepsilon_z}{\partial x²} + \frac{\partial² \varepsilon_x}{\partial z²}=\frac{\partial²\gamma_{xz}}{\partial x \partial z}$$

$$\displaystyle 2 \frac{\partial² \varepsilon_x}{\partial y \partial z}= \frac{\partial}{\partial x}\left( -\frac{\partial \gamma_{yz}}{\partial x} + \frac{\partial \gamma_{xz}}{\partial y} + \frac{\partial \gamma_{xy}}{\partial z}\right)$$

$$\displaystyle 2 \frac{\partial² \varepsilon_y}{\partial x \partial z}= \frac{\partial}{\partial x}\left( \frac{\partial \gamma_{yz}}{\partial x} - \frac{\partial \gamma_{xz}}{\partial y} + \frac{\partial \gamma_{xy}}{\partial z}\right)$$

$$\displaystyle 2 \frac{\partial² \varepsilon_z}{\partial x \partial y}= \frac{\partial}{\partial x}\left( \frac{\partial \gamma_{yz}}{\partial x} + \frac{\partial \gamma_{xz}}{\partial y} - \frac{\partial \gamma_{xy}}{\partial z}\right)$$